# Train SVD for recommandation system
based on Goodread database's rating

We're only using rating since it looks to be the most valuable information we have to determinate the best books a user may want to read.\
In implamentation, we will combine the results with some filtering of our own to give a user what we think is the list of most recommanded books.

## data import

In [37]:
import pandas as pd

r = pd.read_csv('data/ratings.csv')
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


Size reduction is necessary later to visualize cross validation results and save ressources.\
Random state is fixed for better reusability.\
this can be used as r if you want to train the model faster too.

In [38]:
rsample = r.sample(frac=0.1, random_state=42)
rsample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597648 entries, 3623535 to 4905054
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  597648 non-null  int64
 1   book_id  597648 non-null  int64
 2   rating   597648 non-null  int64
dtypes: int64(3)
memory usage: 18.2 MB


## model intialisation
import SVD using previously determined best hyperparameters.\
(look into projectviz.ipynb)

In [39]:
from surprise import SVD
from surprise import NormalPredictor, Dataset, Reader
from surprise.model_selection import cross_validate

# A reader is necessary while using another dataset than provided by surprise.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to 'user id', 'item id' and 'rating' (in that order).
data = Dataset.load_from_df(r[['user_id', 'book_id', 'rating']], reader)

algo = SVD(n_epochs=20, lr_all=0.01, reg_all=0.2, verbose=True) # Verbose is used to visualize epochs

In [29]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.15)

## model training

score visalisation of model using cross validation

In [40]:
datasample = Dataset.load_from_df(rsample[['user_id', 'book_id', 'rating']], reader) # Here we are using a sample 
algosample = SVD(n_epochs=20, lr_all=0.01, reg_all=0.2) # Not using verbose for better vizualisation

cross_validate(algosample, datasample, cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8996  0.8990  0.8983  0.8990  0.0005  
MAE (testset)     0.7134  0.7134  0.7111  0.7126  0.0011  
Fit time          27.92   27.93   28.20   28.02   0.13    
Test time         5.99    8.05    6.05    6.70    0.96    


{'fit_time': (27.920334815979004, 27.927778959274292, 28.200330018997192),
 'test_mae': array([0.71340877, 0.71340772, 0.71108769]),
 'test_rmse': array([0.89960682, 0.8989656 , 0.89833581]),
 'test_time': (5.9923179149627686, 8.05149245262146, 6.0539870262146)}

actual model training

In [42]:
from surprise import accuracy

model = algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


## Saving model

In [47]:
from pickle5 import pickle

# Save to file in the current working directory
pkl_filename = "SVDmodel.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)